# Seasonal Forecast Statistics: Global timeseries

## Import libraries

Switch warnings off for better readability.

In [ ]:
import warnings

from c3s_eqc_automatic_quality_control import diagnostics, download, plot

warnings.filterwarnings("ignore")

## Define request

In [ ]:
collection_id = "seasonal-monthly-single-levels"
year_start = 2015
year_stop = 2015
request = {
    "format": "grib",
    "originating_centre": "cmcc",
    "system": "3",
    "variable": "2m_temperature",
    "product_type": "monthly_mean",
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "leadtime_month": "1",
    "month": [f"{month:02d}" for month in range(1, 13)],
}

## Define transform function

Weighted spatial mean removing leadtime dimension

In [ ]:
def transform_func(ds):
    return diagnostics.spatial_weighted_mean(ds.squeeze("leadtime"))

## Compute spatially-weighted global mean timeseries

In [ ]:
ds_mean = download.download_and_transform(
    collection_id, request, transform_func=transform_func, split_all=True
)

## Plot and save figure

In [ ]:
filename = "_".join(["global-timeseries", collection_id])
title = filename.replace("-", " ").replace("_", " ").title()
fig = plot.shaded_std("t2m", ds_mean, hue_dim="realization", title=title)
fig.show()
fig.write_image(filename + ".png")